# Consumo de la API de Speech-to-Text (Jarvis)

Este proyecto convierte lo que capta tu microfono y lo manda a un servicio de LUIS 

- [Ve la clase grabada aquí](https://web.microsoftstream.com/video/8d9bc7c5-853d-4dbd-ae6c-6b5b504bee59)

Lo que vas a necesitar:
- [Servicio de LUIS](https://www.luis.ai/)
- Servicio de Logic Apps

**Nota:** Si no posees de los dispositivos IoT necesarios puedes modificar el código y llegar hasta obtener la intención y/o acción del servicio de LUIS

Importamos las librerías necesarias.
Aquí vamos a usar el SDK de Cognitive Services para obtener la voz del microfono y transformarlo a texto
- [Referencias del SDK de Python](https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/get-started-speech-to-text?tabs=windowsinstall&pivots=programming-language-python)

In [ ]:
import azure.cognitiveservices.speech as speechsdk
import requests
import json

Posteriormente debes colocar la información necesaria para que funcione el servicio

In [ ]:

url = "URL_DE_CONSUMO_LA_LOGIC_APPS"
header = {"Content-Type": "application/json"}

speech_config = speechsdk.SpeechConfig(subscription="KEY_DEL_SERVICIO_SPEECH_TO_TEXT", region="REGION_SERVICIO")
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, language="es-MX")
result = speech_recognizer.recognize_once()

Se obtiene la respuesta del Servicio de Speech y se crea la condición para cada caso que pueda arrojar el servicio

In [ ]:
if result.reason == speechsdk.ResultReason.RecognizedSpeech:
    print("Recognized: {}".format(result.text))
    # Este es el fragmento de códigi que lo manda a la Logic App
    body = {'\"utterance\"': '\"' + result.text + '\"'}
    x = requests.post(url, json = body, headers = header)
    print(x.json())
    # Aquí termina el fragmento de códigi que lo manda a la Logic App

# Si la IA no reconoce la frase, se muestra un mensaje de error
elif result.reason == speechsdk.ResultReason.NoMatch:
    print("No speech could be recognized: {}".format(result.no_match_details))
# Si el reconocimiento de voz se cancela por alguna razon, se muestra un mensaje de error
elif result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = result.cancellation_details
    print("Speech Recognition canceled: {}".format(cancellation_details.reason))
    # Detalla los mensajes de error
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        print("Error details: {}".format(cancellation_details.error_details))